## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
import tensorflow as tf
import pandas as pd

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default

### Collect Data

In [2]:
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
import pandas as pd
import os

In [5]:
os.chdir('/content/drive/My Drive/01)_GL-AIML/09)_Neural_Networks_&_Deep_Learning/1)_Lab-Internal')
print(os.getcwd())
!ls


/content/drive/My Drive/01)_GL-AIML/09)_Neural_Networks_&_Deep_Learning/1)_Lab-Internal
11_Iris.csv  prices.csv  R6_Internal_Lab_UpdatedTF2_Prices_Iris.ipynb


In [0]:
data = pd.read_csv('prices.csv')

### Check all columns in the dataset

In [7]:
columns = data.columns
print(columns)

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')


### Drop columns `date` and  `symbol`

In [0]:
data = data.drop(['date','symbol'], axis=1)


In [17]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


In [18]:
data.shape

(851264, 5)

### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
data['volume'] = data['volume']/1000000

In [20]:
data.head(8)

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086
5,115.510002,115.550003,114.500000,116.059998,1.0980
6,116.459999,112.849998,112.589996,117.070000,0.9496
7,113.510002,114.379997,110.050003,115.029999,0.7853


In [21]:
data_new = data[0:1000]
data_new.shape

(1000, 5)

### Divide the data into train and test sets

In [22]:
from sklearn.model_selection import train_test_split
import numpy as np
np.random.seed(7)                                    # fix random seed for reproducibility

X = data_new.drop('volume', axis=1)                  # split into input (X) and output (Y) variables.
Y = data_new[['volume']]                             # split into input (X) and output (Y) variables.

train_x,test_x,train_y,test_y=train_test_split(X,Y,test_size=.30,random_state=2)

print (train_x.shape, train_y.shape)
print (test_x.shape, test_y.shape)

(700, 4) (700, 1)
(300, 4) (300, 1)


#### Convert Training and Test Data to numpy float32 arrays


In [0]:
train_x =np.array(train_x).astype('float32')
test_x = np.array(test_x).astype('float32')
train_y =np.array(train_y).astype('float32')
test_y = np.array(test_y).astype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer

transformer = Normalizer()
train_x = transformer.fit_transform(train_x)
test_x = transformer.transform(test_x)

In [25]:
train_x.shape

(700, 4)

In [26]:
train_y.shape

(700, 1)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
import tensorflow as tf
#We are initializing weights and Bias of (Input-Hidden Layer)
w1 = tf.random_normal(shape=(4,3))
b1 = tf.zeros(shape=(3))

#We are initializing weights and Bias of (Hidden-Output)
w2 = tf.random_normal(shape=(3,1))
b2 = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
def prediction(x, w1, b1,w2,b2):
    
    xw_matmul = tf.matmul(x, w1)
    net1 = tf.add(xw_matmul, b1)
    y=tf.sigmoid(net1)                             #JB : Intermidiate/ hidden o/p.
    net2=tf.matmul(y, w2)+b2
    out=tf.sigmoid(net2)                           #JB : Final O/P.
    return net2,out

3.Loss (Cost) Function [Mean square error]

In [0]:
def train(train_x, train_y, w1, b1,w2,b2, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w1,b1,w2,b2])
        
        net2,current_prediction = prediction(train_x, w1, b1,w2,b2)
        current_loss =tf.nn.sigmoid_cross_entropy_with_logits(labels = train_y,logits =net2 )
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw1,db1,dw2,db2 = t.gradient(current_loss,[w1, b1,w2,b2])               
    #JB : Internally Its calculating dE/dw. 
    
    #Update Weights at output layer. 
    w2 = w2 - learning_rate*dw2
    b2 = b2 - learning_rate*db2
     
     #Update Weights at hidden layer
    w1 = w1 - learning_rate*dw1
    b1 = b1 - learning_rate*db1
    
#JB :Refer DL_Model_Regression_Tensorflow for Single line code to calculate weight updation at output & hidden layers.  
    
    return w1, b1,w2,b2

In [30]:
w1

<tf.Tensor 'random_normal:0' shape=(4, 3) dtype=float32>

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
# Refer above line for Code, executed successfully.

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [31]:
for i in range(1000):
    
    w1, b1,w2,b2 = train(train_x, train_y, w1, b1,w2,b2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


### Get the shapes and values of W and b

In [32]:
w1

<tf.Tensor 'sub_3998:0' shape=(4, 3) dtype=float32>

In [33]:
b1

<tf.Tensor 'sub_3999:0' shape=(3,) dtype=float32>

In [34]:
w2

<tf.Tensor 'sub_3996:0' shape=(3, 1) dtype=float32>

In [35]:
b2

<tf.Tensor 'sub_3997:0' shape=(1,) dtype=float32>

### Model Prediction on 1st Examples in Test Dataset

In [36]:
_,y_pred=prediction(test_x,w1,b1,w2,b2)  #JB : testing after succssful completion of defined epox.
out=[]
for val in y_pred:
  if val>np.mean(y_pred):
    out.append(1)
  else:
    out.append(0)
print(out)

OperatorNotAllowedInGraphError: ignored

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [37]:
data_iris = pd.read_csv('11_Iris.csv')
data_iris.head(8)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
5,6,5.4,3.9,1.7,0.4,Iris-setosa
6,7,4.6,3.4,1.4,0.3,Iris-setosa
7,8,5.0,3.4,1.5,0.2,Iris-setosa


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [38]:
data_iris_new = pd.get_dummies(data_iris,prefix=['Species'])
data_iris_new.head(8)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0
5,6,5.4,3.9,1.7,0.4,1,0,0
6,7,4.6,3.4,1.4,0.3,1,0,0
7,8,5.0,3.4,1.5,0.2,1,0,0


### Splitting the data into feature set and target set

In [39]:
# split into input (X) and output (Y) variables
X = data_iris.drop('Species', axis=1)
Y = data_iris[['Species']]
train_x,test_x,train_y,test_y=train_test_split(X,Y,test_size=.30,random_state=2)
print (train_x.shape, train_y.shape)
print (test_x.shape, test_y.shape)
print (X)
print (Y)

(105, 5) (105, 1)
(45, 5) (45, 1)
      Id  SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm
0      1            5.1           3.5            1.4           0.2
1      2            4.9           3.0            1.4           0.2
2      3            4.7           3.2            1.3           0.2
3      4            4.6           3.1            1.5           0.2
4      5            5.0           3.6            1.4           0.2
..   ...            ...           ...            ...           ...
145  146            6.7           3.0            5.2           2.3
146  147            6.3           2.5            5.0           1.9
147  148            6.5           3.0            5.2           2.0
148  149            6.2           3.4            5.4           2.3
149  150            5.9           3.0            5.1           1.8

[150 rows x 5 columns]
            Species
0       Iris-setosa
1       Iris-setosa
2       Iris-setosa
3       Iris-setosa
4       Iris-setosa
..              ..

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
# define baseline model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(8, input_dim=4, activation='relu'))
	model.add(Dense(3, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
	return model

### Model Training 

In [47]:

model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=50, batch_size = train_x.shape[0])
#model.fit(train_X, train_Y, epochs=50)

NameError: ignored

### Model Prediction

### Save the Model

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?